Importation des bibliothèques nécessaires, et calcul des caractéristiques du modèle. Attention : NE PAS EFFACER !

In [ ]:
from pylab import *    # charge toutes les bibliothèques scientifiques
from scipy import interpolate
from scipy.optimize import curve_fit    # permet de modéliser selon un modèle personnalisé
from lmfit import minimize,Parameters,Parameter,report_fit
from lmfit.models import ExpressionModel
from time import *
import re
import numpy as np
#import matplotlib.pyplot as plt
import serial
import ipywidgets as widgets
from IPython.display import display, clear_output

In [6]:
def tableurVersVariables(fichier, delimiter=','):
    tableau = genfromtxt(fichier, delimiter=delimiter, skip_header=0, names=True)    # skip_header permet de ne pas lire les n premières lignes du fichier (utile si il y a des en-têtes) ; delimiter est le signe séparant les valeurs dans les différentes colonnes : par exemple, on peut parfois mettre delimiter=None si les séparateurs sont des espaces
    for i in tableau.dtype.names:    # pour chaque nom détecté en en-tete de colonnes
        globals()[i] = tableau[i]    # on crée une variable qui porte le meme nom et on y attribue les valeurs de la colonne

Paramètres graphiques

In [7]:
# Si on veut différents graphiques interactifs dans la feuille, remplacer inline par ipympl dans la ligne suivante
%matplotlib inline
rcParams['figure.figsize'] = [16, 8]    # pour une fenetre graphique adaptée
rcParams['font.size'] = 15    # pour un texte plus grand
rcParams['lines.markersize'] = 15    # pour des symboles plus grands
rcParams['lines.markeredgewidth'] = 2    # pour des symboles plus larges

Calculs des modélisations

In [8]:
# Calculs du modèle. Attention : NE JAMAIS MODIFIER LES LIGNES SUIVANTES !
def Modele(expression, x, y, contraintes):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)

    for j in contraintes :
        if j[0] in parametres :
            modele.set_param_hint(j[0], value = j[1], vary = j[2], min = j[3], max = j[4])    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    parametres = modele.make_params()
    
    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value:.3g} ; incertitude : {resultat.params[key].stderr:.2g}\n"
    return(modele, resultat.params, valeurs, expression)

def Calcul_modele(abscisse,  ordonnee, equation, debut, fin, debutCourbe, finCourbe, contraintes):
    ## On commence par arranger les expressions avec "x" comme variable, car lmfit travaille avec x par défaut :
    ord = ordonnee
    eq = equation
    equation = re.sub(r"\b"+abscisse+r"\b","x", equation)
    abscisse = globals()[abscisse]
    ordonnee = globals()[ordonnee]
    if debutCourbe == None :
        debutCourbe = min(abscisse)
    if finCourbe == None :
        finCourbe = max(abscisse)
    
    # On calcule maintenant les grandeurs liées au modèle :
    xMod = linspace(debutCourbe, finCourbe, 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)
    modele, parametres, valeurs, expression = Modele(equation, abscisse[debut:fin], ordonnee[debut:fin], contraintes)    # Expression du modèle, et variables en abscisse et en ordonnée ; on peut changer les plages de valeurs à utiliser pour le calcul
    expression = f"{ord} = {eq}"
    yMod = modele.eval(parametres, x = xMod)
    
    # Les paramètres sont définis de façon globale, et utilisables dans des calculs :
    for key in parametres:
        globals()[key] = parametres[key].value
    
    return(xMod, yMod, expression, valeurs, abscisse, ordonnee, modele, parametres)